# T3.2 Analizar e visualizar estatísticas de datos baixados de MongoDB
## Conexión a MongoDB dende Python e análise de datos

En Atlas mete o dataset de exemplo na túa base de datos.

Engade o enderezo IP autorizado a conectar para que poidas conectar á base de datos.

Proba a conectar dende compass coa URL de conexión que che da.

Da BBDD de proba: sample_mfix, na colección movies: https://www.mongodb.com/docs/atlas/sample-data/sample-mflix/

In [30]:
#!conda install -y pymongo dnspython

Conecta a Atlas dende Python, carga a colección en Pandas. Averigua o modo máis adecuado.

In [31]:
from pymongo import MongoClient
import pandas as pd

#HOST="(...).mongodb.net"
HOST="diego.hdjanr7.mongodb.net"
PORT=27017
USERNAME="xuwira05"
PASSWORD="abc123."

if HOST == 'localhost':
    if not USERNAME:
        cli_mongo = MongoClient(HOST, PORT)
    else:
        cli_mongo = MongoClient(HOST, PORT, USERNAME, PASSWORD)
else:
    cli_mongo = MongoClient(f"mongodb+srv://{USERNAME}:{PASSWORD}@{HOST}/?retryWrites=true&w=majority&appName=Diego")

movies = cli_mongo['sample_mflix']['movies']
df = pd.DataFrame(movies.find({}))

A) Contar o total de películas.


In [32]:
len(df)

21349

B) Contar o número de películas de cada xénero.

In [33]:
df.explode('genres')['genres'].value_counts()

genres
Drama          12385
Comedy          6532
Romance         3318
Crime           2457
Thriller        2454
Action          2381
Adventure       1900
Documentary     1834
Horror          1470
Biography       1269
Family          1249
Mystery         1139
Fantasy         1055
Sci-Fi           958
Animation        912
History          874
Music            780
War              699
Short            442
Musical          440
Sport            366
Western          242
Film-Noir         77
News              44
Talk-Show          1
Name: count, dtype: int64

C) Contar cantas películas hai por ano.

In [34]:
df.groupby('year').size()

year
1896     2
1903     1
1909     1
1911     2
1913     1
        ..
2009è    2
2010è    4
2011è    2
2012è    3
2014è    2
Length: 130, dtype: int64

D) Mirar se hai correlación entre imdb e rotten tomatoes.

In [104]:
#imdb = [valor if isinstance(valor,float) else None for dic in df['imdb'] for valor in dic.values()]
#tomatoes=pd.DataFrame(movies.find({},{"tomatoes":{"viewer":{"rating":1}}}))['tomatoes'].dropna()
#tomatoes = [rating*2 for dic in tomatoes for value in dic.values() for rating in value.values()]
#
#dfR=pd.concat([pd.Series(imdb),pd.Series(tomatoes)],keys=['IMDB','TOMATOES'],axis=1)
#dfR.corr()

,IMDB,RATINGS
IMDB,1.00000,0.02882
RATINGS,0.02882,1.00000


In [128]:
imdb = [valor if isinstance(valor,float) else None for dic in df['imdb'] for valor in dic.values()]
tomatoes=[list(value.values())[0] for value in movies.aggregate([{"$match": { "tomatoes.viewer.rating": { "$exists": "true" } }},{"$project":{"_id":0, "items":"$tomatoes.viewer.rating"}}])]

dfR=pd.concat([pd.Series(imdb),pd.Series(tomatoes)],keys=['IMDB','TOMATOES'],axis=1)
dfR.corr()

,IMDB,TOMATOES
IMDB,1.00000,0.02882
TOMATOES,0.02882,1.00000


E) Contar cantas películas ten cada director

In [ ]:
pd.DataFrame(df.explode('directors')['directors'].value_counts())

,count
directors,
Woody Allen,40
Martin Scorsese,32
Takashi Miike,31
Sidney Lumet,29
Steven Spielberg,29
...,...
Cècile Telerman,1
Jeeva,1
Larry Bishop,1


F) Contar en cantas películas ten participado cada actor

In [ ]:
pd.DataFrame(df.explode('cast')['cast'].value_counts())

,count
cast,
Gèrard Depardieu,67
Robert De Niro,58
Michael Caine,51
Bruce Willis,49
Samuel L. Jackson,48
...,...
Margrèt èkadèttir,1
Sibylle Canonica,1
Stefanie von Pfetten,1


G) Contar cantas películas hai de cada idioma

In [ ]:
pd.DataFrame(df.explode('languages')['languages'].value_counts())

,count
languages,
English,14639
French,2405
Spanish,1600
German,1377
Italian,1228
...,...
Quenya,1
Uighur,1
Visayan,1


H) Cal é a media de premios das películas

In [ ]:
dfP = pd.DataFrame([dic for dic in df['awards']])
dfP['wins'].sum()/len(dfP)

4.063047449529252

J) Amosar as películas que teñen alomenos 3 premios

In [ ]:
df[df['awards'].str.get('wins')>=3]

,_id,plot,genres,runtime,cast,poster,title,fullplot,languages,released,...,awards,lastupdated,year,imdb,countries,type,tomatoes,num_mflix_comments,writers,metacritic
44,573a1391f29313caabcd883d,A film about the French Field Marshal's youth ...,"[Biography, Drama, History]",240.0,"[Albert Dieudonnè, Vladimir Roudenko, Edmond V...",https://m.media-amazon.com/images/M/MV5BOGY4M2...,Napoleon,"A massive six-hour biopic of Napoleon, tracing...",NaN,1929-02-17,...,"{'wins': 3, 'nominations': 0, 'text': '3 wins.'}",2015-08-21 00:07:08.593000000,1927,"{'rating': 7.4, 'votes': 5064, 'id': 18192}",[France],movie,"{'viewer': {'rating': 4.3, 'numReviews': 1543,...",0,[Abel Gance],NaN
46,573a1391f29313caabcd89aa,"Two young men, one rich, one middle class, who...","[Drama, Romance, War]",144.0,"[Clara Bow, Charles 'Buddy' Rogers, Richard Ar...",https://m.media-amazon.com/images/M/MV5BYTI1Yj...,Wings,Two young men fall in love with the same same ...,[English],1929-01-05,...,"{'wins': 5, 'nominations': 1, 'text': 'Won 2 O...",2015-09-02 00:36:34.413000000,1927,"{'rating': 7.8, 'votes': 7196, 'id': 18578}",[USA],movie,"{'viewer': {'rating': 3.8, 'numReviews': 3316,...",1,"[John Monk Saunders (story), Hope Loring (scre...",NaN
68,573a1391f29313caabcd93aa,A cabaret singer and a Legionnaire fall in lov...,"[Romance, Drama]",92.0,"[Gary Cooper, Marlene Dietrich, Adolphe Menjou...",https://m.media-amazon.com/images/M/MV5BOWNiYz...,Morocco,The Foreign Legion marches in to Mogador with ...,"[English, French, Spanish, Arabic, Italian]",1930-12-06,...,"{'wins': 7, 'nominations': 0, 'text': 'Nominat...",2015-08-12 00:08:59.097000000,1930,"{'rating': 7.3, 'votes': 3566, 'id': 21156}",[USA],movie,"{'viewer': {'rating': 3.6, 'numReviews': 1303,...",0,"[Jules Furthman (adapted by), Benno Vigny (fro...",NaN
69,573a1391f29313caabcd944c,Young Harry is in love and wants to marry an a...,"[Drama, Romance]",76.0,"[Greta Garbo, Lewis Stone, Gavin Gordon, Ellio...",https://m.media-amazon.com/images/M/MV5BYTc4Yj...,Romance,Young Harry is in love and wants to marry an a...,[English],1930-08-26,...,"{'wins': 3, 'nominations': 0, 'text': 'Nominat...",2015-08-08 00:38:18.023000000,1930,"{'rating': 6.0, 'votes': 379, 'id': 21310}",[USA],movie,"{'viewer': {'rating': 3.5, 'numReviews': 191, ...",0,"[Edward Sheldon (from the play by), Bess Mered...",NaN
74,573a1391f29313caabcd95f6,A newspaper editor settles in an Oklahoma boom...,"[Drama, Western]",123.0,"[Richard Dix, Irene Dunne, Estelle Taylor, Nan...",https://m.media-amazon.com/images/M/MV5BMjExOD...,Cimarron,When the government opens up the Oklahoma terr...,"[English, French]",1931-02-09,...,"{'wins': 5, 'nominations': 4, 'text': 'Won 3 O...",2015-08-21 00:50:48.813000000,1931,"{'rating': 6.0, 'votes': 3310, 'id': 21746}",[USA],movie,"{'viewer': {'rating': 2.8, 'numReviews': 1422,...",1,"[Edna Ferber (novel), Howard Estabrook (screen...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21304,573a13f8f29313caabde8161,NaN,NaN,120.0,"[Aharon Traitel, Khalifa Natour, Riki Blich, G...",NaN,Tikkun,NaN,"[Hebrew, Yiddish]",2015-07-10,...,"{'wins': 7, 'nominations': 1, 'text': '7 wins ...",2015-09-04 00:39:57.163000000,2015,"{'rating': 7.0, 'votes': 23, 'id': 4482858}",[Israel],movie,NaN,0,NaN,NaN
21317,573a13f9f29313caabdea63e,"Three different love stories, set in three con...",[Drama],123.0,"[Tihana Lazovic, Goran Markovic, Nives Ivankov...",https://m.media-amazon.com/images/M/MV5BMTQ4ND...,The High Sun,"Three different love stories, set in three con...",[Croatian],2015-05-17,...,"{'wins': 8, 'nominations': 3, 'text': '8 wins ...",2015-08-21 00:54:49.047000000,2015,"{'rating': 7.8, 'votes': 161, 'id': 4593108}","[Croatia, Serbia, Slovenia]",movie,NaN,0,[Dalibor Matanic],NaN
21330,573a13f9f29313caabdeb320,For anyone who has not fully understood the co...,"[Documentary, News]",60.0,"[John Barrett, Andreas M. Antonopoulos, Jeffre...",https://m.media-amazon.com/images/M/MV5BNjU3MT...,Bitc

K) Contar as películas por país

In [ ]:
pd.DataFrame(df.explode('countries')['countries'].value_counts())

,count
countries,
USA,10921
UK,2652
France,2647
Germany,1494
Canada,1260
...,...
Zaire,1
Trinidad and Tobago,1
Martinique,1
